<a href="https://colab.research.google.com/github/bunny-ml/web_scraping/blob/main/WebScrapingScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np
import csv
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
import random
import re
import time

In [3]:
all_product_names = []
all_product_prices = []
all_product_ratings = []

# header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.35'}
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}

for page_num in range(1 , 20):
  url = f'https://www.amazon.in/s?k=buget+laptop&page={page_num}'
  response = requests.get(url,headers=header)
  soup = BeautifulSoup(response.text,'html.parser')

  product_names =[a.text for a in soup.find_all('span' , {'class' : 'a-size-medium a-color-base a-text-normal'})]
  product_prices =[a.text for a in soup.find_all('span' , {'class' : 'a-price-whole'})]
  product_ratings =[a.text for a in soup.find_all('span' , {'class' : 'a-icon-alt'})]



  all_product_names.extend(product_names)
  all_product_prices.extend(product_prices)
  all_product_ratings.extend(product_ratings)

  time.sleep(3)



In [4]:
def first_word(company_name):
  comp_name = company_name.split()
  if len(comp_name) > 0:
    if comp_name[0] == "Refurbished" or comp_name[0] == "Renewed":
        return comp_name[1]
    else:
      return comp_name[0]
  else:
    return "Unknown"

def first_word1(company_rating):
    ratings = company_rating.split()
    if len(ratings) > 0:
      return ratings[0]
    else:
      return None


def extract_processor(product_name):
  match = re.search(r'(Intel|AMD|Ryzen|M1|Apple)[\s\w]+', product_name)
  if match:
    return match.group(0).strip()
  else:
    return None

def extract_gpu(product_name):
  match = re.search(r'(NVIDIA|RTX|GeForce|GTX|AMD Radeon|RX)[\s\w]+', product_name)
  if match:
    return match.group(0).strip()
  else:
    return None

In [5]:
min_length = min(len(all_product_names) , len(all_product_prices))

all_product_prices = all_product_prices[:min_length]
all_product_names = all_product_names[:min_length]

df = pd.DataFrame(
    {
        "product_name": all_product_names,
        "product_price": all_product_prices

    }
)
df['potential_company'] = df['product_name'].apply(first_word)
df['product_rating'] = pd.Series(all_product_ratings).apply(first_word1)
df['processor'] = df['product_name'].apply(extract_processor)
df['gpu'] = df['product_name'].apply(extract_gpu)

In [15]:
df

,product_name,product_price,potential_company,product_rating,processor,gpu
0,NaN,NaN,NaN,4.1,NaN,NaN
1,NaN,NaN,NaN,3.0,NaN,NaN
2,NaN,NaN,NaN,4.3,NaN,NaN
3,NaN,NaN,NaN,Previous,NaN,NaN
4,NaN,NaN,NaN,Next,NaN,NaN
5,NaN,NaN,NaN,5.0,NaN,NaN
6,NaN,NaN,NaN,3.4,NaN,NaN
7,NaN,NaN,NaN,3.8,NaN,NaN
8,NaN,NaN,NaN,4.4,NaN,NaN
9,NaN,NaN,NaN,4.0,NaN,NaN


In [12]:
df.isnull().sum()

,0
product_name,20
product_price,20
potential_company,20
product_rating,0
processor,20
gpu,20


In [14]:
df.shape

(20, 6)

In [ ]:
# Create a new column 'refurbished' and initialize it with False
df['refurbished'] = False

# Iterate over the DataFrame and check if 'Refurbished' is in the product name
for index, row in df.iterrows():
  if 'Refurbished' in row['product_name'] or 'Renewed' in row['product_name']:
    df.loc[index, 'refurbished/renewed'] = True
    df.loc[index, 'product_name'] = row['product_name'].replace('[Refurbished]', '').replace('(Renewed)', '').strip()



TypeError: argument of type 'float' is not iterable

In [ ]:
df.info()

In [ ]:
df.head(100)